In [1]:
import os
import pandas as pd
dir_percent = int(input("Enter Direct attainment percentage "))
indir_percent = int(input("Enter Indirect attainment percentage "))
subject_files = []
matrix_file_name = ""
for i in os.listdir():
    if "CO PO PSO" in i:
        matrix_file_name  = i
    if "Course End" not in i and "Assignment" not in i and "UNIV" not in i and "MODEL" not in i and "FIAT" not in i and "SIAT" not in i and ".xls" in i and len(i)<25:
        subject_files.append(i)

Enter Direct attainment percentage 90
Enter Indirect attainment percentage 10


In [ ]:
def write_final_co(excel_file_name):
    df_Paper = pd.read_excel(excel_file_name)
    from xlrd import open_workbook
    from xlutils.copy import copy
    rb = open_workbook(excel_file_name)
    wb = copy(rb)
    s = wb.get_sheet(0)
    length = len(df_Paper.loc[:,'Direct CO Attainment Level'])
    dir_co_att,indir_co_att = [],[]
    for i in df_Paper.loc[:,'Direct CO Attainment']:
        if pd.isna(i):
            break
        dir_co_att.append(i)
    for i in df_Paper.loc[:,'Indirect CO Attainment']:
        if pd.isna(i):
            break
        indir_co_att.append(i)
    final_co_att = [ ((dir_co_att[i])*dir_percent/100) + ((indir_co_att[i])*indir_percent/100)
                        for i in range(len(dir_co_att))]
    
    for i in range(len(dir_co_att)):
        s.write(i+1,9,final_co_att[i])
    ctr = 0
    for i in range(len(dir_co_att)):
        if final_co_att[ctr] >= 60:
            s.write(i+1,10,3)
        elif final_co_att[ctr] >= 50:
            s.write(i+1,10,2)
        elif final_co_att[ctr] >= 40:
            s.write(i+1,10,1)
        else:
            s.write(i+1,10,0)
        ctr+=1
    
    wb.save(excel_file_name)
    

In [ ]:
def get_CO_mapping(subject_name):
    df_CO = pd.read_excel(matrix_file_name,sheet_name="Sheet1")
    df_CO.head()
    index_of_subject = 0
    for i in range(len(df_CO)):
        if str(df_CO.iloc[i,0]) == subject_name:
            index_of_subject = i
            break
    whole_List = []
    for ctr in range(1,10):
        CO_List = []
        try:
            for i in df_CO.iloc[index_of_subject+ctr,:]:
                if pd.isna(i):
                    break
                CO_List.append(i)
        except:
            pass
        if len(CO_List) <= 1 :break
        whole_List.append(CO_List)
    return whole_List

In [ ]:
def get_list(k,check_list,po_matrix):
    temp = []
    for i in range(len(po_matrix)):
        for j in range(len(po_matrix[0])):
            if j==k:
                temp.append(po_matrix[i][j+1])
    return temp

In [ ]:
def write_co_tabel(excel_file_name,po_matrix,check_list):
    df_Paper = pd.read_excel(excel_file_name)
    total_length = len(df_Paper.loc[:,])
    from xlrd import open_workbook
    from xlutils.copy import copy
    rb = open_workbook(excel_file_name)
    wb = copy(rb)
    s = wb.get_sheet(0)
    length = len(df_Paper.loc[:,'Direct CO Attainment Level'])
    co_att = []
    length = 0
    for i in df_Paper.loc[:,'Final CO Attainment Level']:
        if pd.isna(i):
            break
        co_att.append(i)
        length+=1
    s.write(total_length+2,0,'CO#')
    s.write(total_length+2,1,'CO Attainment')
    s.write(total_length+5+length,0,'CO#')
    s.write(total_length+5+length,1,'CO Attainment')
    for i in range(length):
        s.write(total_length+3+i,0,'CO'+str(i+1))
        s.write(total_length+3+i,1,co_att[i])
        s.write(total_length+6+i+length,0,'CO'+str(i+1))
        s.write(total_length+6+i+length,1,co_att[i])
    counter = 2
    for i in range(len(check_list[:-3])):
        if check_list[i] == 1:
            s.write(total_length+2,counter,"Weightage")
            s.write(total_length+2,counter+1,"PO"+str(i+1)+" Attainment")
            counter+=2
    counter = 2
    for i in range(len(check_list[-3:])):
        if check_list[i+len(check_list[:-3])] == 1:
            s.write(total_length+5+length,counter,"Weightage")
            s.write(total_length+5+length,counter+1,"PSO"+str(i+1)+" Attainment")
            counter+=2
    col = 2
    row_adjustment = 0
    if total_length == 5:
        row_adjustment = 1
    for k in range(len(check_list[:-3])):
        if check_list[k] == 1:        
            recieved_list = get_list(k,check_list,po_matrix)
            ctr = 0
            temp_list = []
            last_num = 0
            for i in range(total_length+2,total_length+2+len(co_att)):
                if recieved_list[ctr] == '-':
                    s.write(total_length-5+i+row_adjustment,col,'-')
                    s.write(total_length-5+i+row_adjustment,col+1,"-")
                elif recieved_list[ctr]>=3:
                    s.write(total_length-5+i+row_adjustment,col,1)
                    s.write(total_length-5+i+row_adjustment,col+1,co_att[ctr]*1)
                    temp_list.append(co_att[ctr])
                elif recieved_list[ctr]>=2:
                    s.write(total_length-5+i+row_adjustment,col,0.75)
                    s.write(total_length-5+i+row_adjustment,col+1,co_att[ctr]*0.75)
                    temp_list.append(co_att[ctr]*0.75)
                elif recieved_list[ctr] >= 1:
                    s.write(total_length-5+i+row_adjustment,col,0.5)
                    s.write(total_length-5+i+row_adjustment,col+1,co_att[ctr]*0.5)
                    temp_list.append(co_att[ctr]*0.5)
                else:
                    s.write(total_length-5+i+row_adjustment,col,0)
                    s.write(total_length-5+i+row_adjustment,col+1,0)
                    temp_list.append(0)
                ctr+=1
                last_num = i
            s.write(total_length-5+last_num+1+row_adjustment,col+1,sum(temp_list)/len(temp_list))
            col+=2
        
    col = 2
    row_adjustment = 0
    if total_length == 5:
        row_adjustment = 1
    for k in range(len(check_list[-3:])):
        if check_list[k+len(check_list[:-3])] == 1:        
            recieved_list = get_list(k+len(check_list[:-3]),check_list,po_matrix)
            ctr = 0
            temp_list = []
            last_num = 0
            for i in range(total_length+5+length,total_length+5+len(co_att)+length):
                if recieved_list[ctr] == '-':
                    s.write(total_length-5+i+row_adjustment,col,'-')
                    s.write(total_length-5+i+row_adjustment,col+1,"-")
                elif recieved_list[ctr]>=3:
                    s.write(total_length-5+i+row_adjustment,col,1)
                    s.write(total_length-5+i+row_adjustment,col+1,co_att[ctr]*1)
                    temp_list.append(co_att[ctr])
                elif recieved_list[ctr]>=2:
                    s.write(total_length-5+i+row_adjustment,col,0.75)
                    s.write(total_length-5+i+row_adjustment,col+1,co_att[ctr]*0.75)
                    temp_list.append(co_att[ctr]*0.75)
                elif recieved_list[ctr] >= 1:
                    s.write(total_length-5+i+row_adjustment,col,0.5)
                    s.write(total_length-5+i+row_adjustment,col+1,co_att[ctr]*0.5)
                    temp_list.append(co_att[ctr]*0.5)
                else:
                    s.write(total_length-5+i+row_adjustment,col,0)
                    s.write(total_length-5+i+row_adjustment,col+1,0)
                    temp_list.append(0)
                ctr+=1
                last_num = i
            s.write(total_length-5+last_num+1+row_adjustment,col+1,sum(temp_list)/len(temp_list))
            col+=2
            
            
    wb.save(excel_file_name)


In [ ]:
for i in subject_files:
    write_final_co(i)
    try:
        excel_file_name = i
        temp = i.replace(" ","/")[:-4]
        po_matrix = get_CO_mapping(temp)
        print(temp,end="\n\n")
        print("      PO1  PO2  PO3  PO4  PO5  PO6  PO7  PO8  PO9 PO10 PO11 PO12 PSO1 PSO2  PSO")
        for row in po_matrix:
            print(*row,sep="    ")
        check_list = [0 for i in range(len(po_matrix[0]))]
        for i in range(len(po_matrix)):
            for j in range(1,len(po_matrix[i])):
                if po_matrix[i][j]!='-':
                    check_list[j] = 1
        print("   ",*check_list[1:],sep="    ")
        print("\n--------------------------------------------------------------------------------\n")
        write_co_tabel(excel_file_name,po_matrix,check_list[1:])
    except:
        print("\n\nSkipping "+str(temp)+"...\n\nSome Error occured...\n\nPlease check the file Manually...\n\n")
        print("Data Skipped")
        print(excel_file_name)
        print(po_matrix,sep="\n")
        print(check_list[1:])

In [ ]:
import shutil
output_files = []
try:
    print("\n\nTrying to create Output folder...\n\n")
    os.makedirs("Output")
    print("\n\nOutput Folder created successfully...\n\n")
except:
    print("\n\nOutput folder already exist...\n\n")
for i in os.listdir():
    if ".xls" in i and ".xlsx" not in i:
        output_files.append(i)
for i in output_files:
    print("\nMoving "+str(i)+" to Outputs folder")
    shutil.move(os.path.basename(i),"Output/"+str(os.path.basename(i)))

In [ ]:
print("\nProgram Executed Successfully...\n\nDownload Your Files from Output folder...\n")